In [25]:
import numpy as np
import imageio as iio
import DatasetSplitter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as pca_test


In [26]:
trainingData, trainingLabels, testingData, testingLabels = DatasetSplitter.differentSplitData()
trainingData = np.array(trainingData)
print(trainingData.shape)

IndexError: list index out of range